In [2]:
#dependencies 
#had to 
import pandas as pd 
import numpy as np 
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import config 
from coincallandwritepythonfile import refined_coin_list, injest_date, write_to_storage, convert_dataframe_to_file

In [3]:
#make sure no blank id values for query
refined_coin_list = refined_coin_list[refined_coin_list['id'] != ""]
#get ids for each coin and convert to list
coin_ids = refined_coin_list.id.to_list()

In [4]:
number_of_coin_ids = len(coin_ids)
#max number is 30 per minute for the API
number_of_queries_per_run = 25
rounded_up = np.ceil(number_of_coin_ids/ number_of_queries_per_run)

In [5]:
def Make_Call (id, api_key):
    url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
    parameters = {
        'id': id,
                }
    headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': api_key
    }

    #make API call and return response as a dictionary
    session = Session()
    session.headers.update(headers)
    response = session.get(url, params=parameters)
    data = json.loads(response.text)
    #erroring on 'Data' sometimes, like a rogue id
    #erroring because can't make more than 30 calls per minute 
    return data

In [6]:

import time
  
#schedule.run_all(delay_seconds= 3).do(func)
def get_coin_info_breakup(list_of_coins_, api_key):
    """Function makes API call for each coin ID within list of coins.
    Designed to take list from top 100 coins and get their most recent price information. """

    #create an empty dataframe that we will append responses too
    df = pd.DataFrame()
    #a count variable to index the respones when trying to concat 
    #get parameters for API call
    number_of_coin_ids = len(list_of_coins_)
    #max number is 30 per minute for the API
    number_of_queries_per_run = 25
    rounded_up = np.ceil(number_of_coin_ids/ number_of_queries_per_run)
    coin_array = list_of_coins_
    test = np.array_split(coin_array,rounded_up)
    count =0
    #schedule.every(60).seconds.do(Make_Call(id, api_key))
    while count < rounded_up:
        for id in test[count]:
            try:
                data = Make_Call(id, api_key)
                cmc_df = pd.DataFrame.from_dict(data['data'], orient='index')

                #expand nested quote column containing price information
                expanded_df = pd.DataFrame.from_dict(cmc_df['quote'][0], orient= 'index')     
                #set then index for count ie the number of runs 
                expanded_df.set_index(pd.Index([count]),inplace= True)

                #dropping nested columns that we won't explore during this exercise
                trial = cmc_df.drop(['quote','tags', 'platform'], axis= 1)
                
                #setting index as count variable so can join back
                trial.set_index(pd.Index([count]),inplace= True)
                
                #concat was tricky had to add
                #had to add key to get them to append properly
                coin_price_df = pd.concat([trial,expanded_df], axis=1, keys=['trial', 'expanded_df']  )
                #coin_price_df.reset_index(drop = True, inplace= True)
                coin_price_merge = pd.merge(trial, expanded_df)
                #this saves it as a coreframe

                    
                df = df.append(coin_price_merge) 
            except KeyError:
                continue
            
            #parse dict and load in relavant data to a DF
        time.sleep(60)
        count += 1
    return df 

In [7]:
df = get_coin_info_breakup(coin_ids,  config.coin_api_key)

In [11]:
#write to storage function takes dataset input as a CSV so had to convert
df["File_Name"] = "Coin_Price_Fact"
#Add column for injested at 
df["injest_datetime"] = injest_date
#convert to CSV to write to storage
df_csv = convert_dataframe_to_file(df, "csv")

In [12]:
#write the new DF to storage
storage_account_name = config.storage_account_name
storage_account_key = config.storage_account_key
container_name = "coinprice"
directory_name = "raw"

dataframe = write_to_storage(storage_account_name=storage_account_name, storage_account_key= storage_account_key, container_name= container_name, directory_name= directory_name,\
    dataset = df_csv, file_name= "coinprice.csv")

### Refine dataset to write to refined ADLS

In [14]:
#remove null values
dataframe = dataframe[dataframe.name.notnull()]
price_and_supply_columns = ['id', 'name', 'symbol','price', 'max_supply', 'circulating_supply', \
   'total_supply', 'market_cap','File_Name', 'injest_datetime']
volume_and_percent_change_columns = ['id', 'volume_24h', 'volume_change_24h', 'percent_change_1h', 'percent_change_24h',
       'percent_change_7d', 'percent_change_30d', 'percent_change_60d',
       'percent_change_90d','File_Name', 'injest_datetime']


df_price_and_supply = dataframe[price_and_supply_columns]
df_price_and_supply_csv= convert_dataframe_to_file(df_price_and_supply, "csv")

df_volume_and_percent_change = dataframe[volume_and_percent_change_columns]
df_volume_and_percent_change_csv = convert_dataframe_to_file(df_volume_and_percent_change, "csv")


In [15]:
#write the new DF to storage
storage_account_name = config.storage_account_name
storage_account_key = config.storage_account_key
container_name_price = "coinprice"
directory_name = "refined"

container_name_volume = "coinvolume"

df_volume_and_percent_change_csv
df_price_and_supply_csv

dataframe_price_supply = write_to_storage(storage_account_name=storage_account_name, storage_account_key= storage_account_key,\
     container_name= container_name_price, directory_name= directory_name,\
    dataset = df_price_and_supply_csv, file_name= "coinpricerefined.csv")

dataframe_price_supply = write_to_storage(storage_account_name=storage_account_name, storage_account_key= storage_account_key,\
     container_name= container_name_volume, directory_name= directory_name,\
    dataset = df_volume_and_percent_change_csv, file_name= "coinvolumefined.csv")